# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging and appending 

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Read in the Outcomes data
# Let's be sure to parse dates for the DateTime and Date of Birth columns
outcomes = pd.read_csv("data/Austin_Animal_Center_Outcomes_022822.csv",
                       parse_dates=['DateTime', 'Date of Birth'])

In [5]:
# Check it
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [17]:
outcomes['Time Delta'] = outcomes['DateTime'].dt.normalize() - outcomes['Date of Birth']

In [22]:
outcomes['Time Delta']

0         736 days
1         371 days
2         366 days
3         128 days
4           6 days
            ...   
137092     66 days
137093    736 days
137094    664 days
137095   3058 days
137096   1165 days
Name: Time Delta, Length: 137097, dtype: timedelta64[ns]

In [12]:
# Let's create our Age in Days column
outcomes['Age in Days'] = (outcomes['DateTime'].dt.normalize() - outcomes['Date of Birth']).dt.days

In [16]:
# Grab just the integer here...
outcomes['Age in Days'] 

0          736
1          371
2          366
3          128
4            6
          ... 
137092      66
137093     736
137094     664
137095    3058
137096    1165
Name: Age in Days, Length: 137097, dtype: int64

In [13]:
# Sanity check
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [14]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137097 entries, 0 to 137096
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         137097 non-null  object        
 1   Name              96095 non-null   object        
 2   DateTime          137097 non-null  datetime64[ns]
 3   MonthYear         137097 non-null  object        
 4   Date of Birth     137097 non-null  datetime64[ns]
 5   Outcome Type      137073 non-null  object        
 6   Outcome Subtype   62653 non-null   object        
 7   Animal Type       137097 non-null  object        
 8   Sex upon Outcome  137095 non-null  object        
 9   Age upon Outcome  137092 non-null  object        
 10  Breed             137097 non-null  object        
 11  Color             137097 non-null  object        
 12  Age in Days       137097 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(10)
memory usage: 13

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [23]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby(by='Animal Type')

In [26]:
# But if we add an aggregation function to tell it what to do with the other cols...
outcomes.groupby(by='Animal Type').mean()

,Age in Days
Animal Type,
Bird,533.716981
Cat,532.816939
Dog,1004.114514
Livestock,503.800000
Other,479.143665


## `.groups` and `.get_group()`

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [27]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value
outcomes.groupby(by='Animal Type').groups

{'Bird': [206, 534, 985, 1027, 1284, 1310, 2220, 2258, 2274, 2417, 2521, 2598, 2712, 2778, 3178, 3379, 3648, 3743, 4003, 4024, 4288, 4702, 4766, 4998, 5063, 5205, 5436, 5656, 5848, 6087, 6236, 6340, 6592, 6682, 7033, 7352, 7428, 7985, 8048, 8315, 8331, 8414, 8538, 8922, 9203, 9448, 9758, 9825, 10103, 10165, 10407, 10657, 10736, 11386, 11616, 11674, 11732, 11765, 11771, 11837, 12205, 12418, 12423, 12451, 12474, 12713, 12902, 12978, 13057, 13063, 13095, 13272, 13317, 13323, 13435, 13474, 13677, 13934, 13950, 13963, 13981, 14108, 14131, 14146, 14193, 15114, 15193, 15543, 15553, 15813, 16022, 16197, 16499, 16866, 17173, 17338, 17390, 17426, 18319, 18359, ...], 'Cat': [0, 4, 7, 8, 10, 11, 14, 15, 16, 17, 18, 20, 24, 26, 34, 37, 49, 54, 56, 66, 67, 68, 70, 75, 78, 80, 83, 84, 89, 90, 92, 94, 95, 97, 98, 102, 113, 115, 116, 117, 118, 120, 122, 126, 139, 141, 142, 145, 147, 148, 151, 152, 156, 157, 158, 164, 167, 168, 170, 171, 176, 178, 184, 191, 192, 194, 200, 202, 203, 207, 209, 212, 215, 2

In [30]:
# Once we know the group indices, we can return the groups using those indices
outcomes.groupby(by='Animal Type').get_group('Bird').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days,Time Delta
206,A720727,Rooster 11,2016-03-08 13:47:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Red,388,388 days
534,A720734,Rooster 18,2016-03-08 15:07:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Chocolate,388,388 days
985,A779213,NaN,2018-09-07 11:17:00,Sep 2018,2017-08-27,Adoption,Foster,Bird,Unknown,1 year,Quaker Mix,Green/Silver,376,376 days
1027,A760051,NaN,2017-10-20 12:40:00,Oct 2017,2016-10-11,Adoption,NaN,Bird,Unknown,1 year,Quaker,Green/Gray,374,374 days
1284,A790172,NaN,2019-03-07 00:00:00,Mar 2019,2018-03-05,Euthanasia,Suffering,Bird,Unknown,1 year,Chicken Mix,White/Red,367,367 days


In [32]:
# Why yes, this is the same as
outcomes.loc[outcomes['Animal Type'] == 'Bird'].head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days,Time Delta
206,A720727,Rooster 11,2016-03-08 13:47:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Red,388,388 days
534,A720734,Rooster 18,2016-03-08 15:07:00,Mar 2016,2015-02-14,Adoption,NaN,Bird,Intact Male,1 year,Chicken Mix,Black/Chocolate,388,388 days
985,A779213,NaN,2018-09-07 11:17:00,Sep 2018,2017-08-27,Adoption,Foster,Bird,Unknown,1 year,Quaker Mix,Green/Silver,376,376 days
1027,A760051,NaN,2017-10-20 12:40:00,Oct 2017,2016-10-11,Adoption,NaN,Bird,Unknown,1 year,Quaker,Green/Gray,374,374 days
1284,A790172,NaN,2019-03-07 00:00:00,Mar 2019,2018-03-05,Euthanasia,Suffering,Bird,Unknown,1 year,Chicken Mix,White/Red,367,367 days


### Multi-Indexing

In [36]:
# Same goes for multi-index groupbys
outcomes.groupby(["Animal Type", 'Outcome Type'])

In [46]:
# .groups outputs a dictionary, so we can access the group names using keys()
outcomes.groupby(["Animal Type", 'Outcome Type']).groups.keys()

dict_keys([('Cat', 'Rto-Adopt'), ('Dog', 'Adoption'), ('Other', 'Euthanasia'), ('Cat', 'Transfer'), ('Cat', 'Adoption'), ('Cat', 'Return to Owner'), ('Dog', 'Return to Owner'), ('Dog', 'Transfer'), ('Cat', 'Euthanasia'), ('Other', 'Adoption'), ('Dog', 'Rto-Adopt'), ('Cat', 'Died'), ('Dog', 'Euthanasia'), ('Other', 'Transfer'), ('Bird', 'Adoption'), ('Other', 'Disposal'), ('Other', 'Died'), ('Dog', 'Died'), ('Cat', 'Disposal'), ('Other', 'Return to Owner'), ('Bird', 'Euthanasia'), ('Bird', 'Transfer'), ('Livestock', 'Return to Owner'), ('Dog', 'Missing'), ('Other', 'Relocate'), ('Dog', nan), ('Livestock', 'Adoption'), ('Bird', 'Return to Owner'), ('Dog', 'Disposal'), ('Cat', 'Missing'), ('Bird', 'Disposal'), ('Bird', 'Died'), ('Other', 'Missing'), ('Other', 'Rto-Adopt'), ('Bird', 'Relocate'), ('Bird', 'Missing'), ('Other', nan), ('Livestock', 'Transfer'), ('Cat', 'Relocate'), ('Cat', nan), ('Livestock', 'Died'), ('Livestock', 'Euthanasia')])

In [39]:
# We can then get a specific group, such as cats that were adopted
outcomes.groupby(["Animal Type", 'Outcome Type']).get_group(('Cat', 'Adoption')).head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days,Time Delta
7,A689724,*Donatello,2014-10-18 18:52:00,Oct 2014,2014-08-01,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,78,78 days
8,A680969,*Zeus,2014-08-05 16:59:00,Aug 2014,2014-06-03,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,63,63 days
20,A730621,*Liza,2016-09-10 18:59:00,Sep 2016,2016-05-18,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Calico,115,115 days
26,A801106,NaN,2019-08-16 14:05:00,Aug 2019,2019-05-06,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Orange Tabby,102,102 days
54,A792258,Vesper,2019-04-10 20:53:00,Apr 2019,2016-09-08,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Tortie,944,944 days


## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [40]:
# Let's try it out
outcomes.groupby('Animal Type').count()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days,Time Delta
Animal Type,,,,,,,,,,,,,
Bird,636,145,636,636,636,636,380,636,636,636,636,636,636
Cat,52092,30380,52092,52092,52092,52088,31815,52092,52091,52092,52092,52092,52092
Dog,77091,64516,77091,77091,77091,77076,24590,77089,77089,77091,77091,77091,77091
Livestock,25,3,25,25,25,25,19,25,25,25,25,25,25
Other,7253,1051,7253,7253,7253,7248,5849,7253,7251,7253,7253,7253,7253


## Exercise

Use `.groupby()` to find the most recently born of each (main) animal type.

In [48]:
# Your code here
outcomes.groupby(by='Animal Type')['Age in Days'].min()

Animal Type
Bird            3
Cat          -335
Dog         -1116
Livestock      15
Other           0
Name: Age in Days, dtype: int64

In [55]:
outcomes.groupby(by='Animal Type')['Date of Birth'].max()

Animal Type
Bird        2022-01-06
Cat         2022-02-18
Dog         2022-02-14
Livestock   2020-05-28
Other       2022-02-11
Name: Date of Birth, dtype: datetime64[ns]

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [56]:
# Groupby two columns
outcomes.groupby(by=['Outcome Type', 'Sex upon Outcome']).agg('mean')

Age in Days
Outcome Type    Sex upon Outcome             
Adoption        Intact Female      419.505568
                Intact Male        476.596774
                Neutered Male      651.126203
                Spayed Female      648.050078
                Unknown            389.964286
Died            Intact Female      349.628049
                Intact Male        304.392157
                Neutered Male     1859.732673
                Spayed Female     2099.905263
                Unknown            311.192737
Disposal        Intact Female      407.277778
                Intact Male        735.935484
                Neutered Male     1987.777778
                Spayed Female     2470.166667
                Unknown            447.166016
Euthanasia      Intact Female     1133.424632
                Intact Male        902.476300
                Neutered Male     2226.837838
                Spayed Female     2313.677966
                Unknown            503.333588
Missing         Intact Female      266.800000
                Intact Male        340.846154
                Neutered Male     1188.285714
                Spayed Female     1262.733333
                Unknown            169.250000
Relocate        Intact Female      732.000000
                Neutered Male     1105.000000
                Spayed Female      495.000000
                Unknown            612.200000
Return to Owner Intact Female     1067.053879
                Intact Male       1110.232182
                Neutered Male     1634.609964
                Spayed Female     1751.059214
                Unknown            764.225275
Rto-Adopt       Intact Female     1481.142857
                Intact Male       1503.175000
                Neutered Male     1254.399577
                Spayed Female     1252.076923
                Unknown           1590.000000
Transfer        Intact Female      409.499911
                Intact Male        351.252856
                Neutered Male     1126.408920
                Spayed Female     1097.636076
                Unknown            169.775301

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [57]:
# Now try with a pivot table
outcomes.pivot_table(index='Outcome Type', columns='Sex upon Outcome', aggfunc='mean')

Age in Days                                           \
Sex upon Outcome Intact Female  Intact Male Neutered Male Spayed Female   
Outcome Type                                                              
Adoption            419.505568   476.596774    651.126203    648.050078   
Died                349.628049   304.392157   1859.732673   2099.905263   
Disposal            407.277778   735.935484   1987.777778   2470.166667   
Euthanasia         1133.424632   902.476300   2226.837838   2313.677966   
Missing             266.800000   340.846154   1188.285714   1262.733333   
Relocate            732.000000          NaN   1105.000000    495.000000   
Return to Owner    1067.053879  1110.232182   1634.609964   1751.059214   
Rto-Adopt          1481.142857  1503.175000   1254.399577   1252.076923   
Transfer            409.499911   351.252856   1126.408920   1097.636076   

                               
Sex upon Outcome      Unknown  
Outcome Type                   
Adoption           389.964286  
Died               311.192737  
Disposal           447.166016  
Euthanasia         503.333588  
Missing            169.250000  
Relocate           612.200000  
Return to Owner    764.225275  
Rto-Adopt         1590.000000  
Transfer           169.775301

## Exercise

Use `.pivot_table()` to add up the number of my tasks by category. 

>  Hint: Use `sum()` as your aggregating function.

In [58]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

,category,descr,priority,num_tasks
0,house,kitchen,2,2
1,house,laundry,3,1
2,school,git,4,2
3,school,Python,1,3


In [68]:
# Your code here
tasks.pivot_table(index='category', aggfunc=sum, values='num_tasks')

,num_tasks
category,
house,3
school,5


# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [69]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [70]:
toy2

,age,MP
0,63,100
1,33,200


In [71]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2)

ValueError: columns overlap but no suffix specified: Index(['age'], dtype='object')

In [72]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [73]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


In [74]:
toy1.drop('age', axis=1).join(toy2)

,HP,age,MP
0,142,63,100
1,47,33,200


## `.merge()`

Or we could use `.merge()`:

In [77]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [78]:
# Let's try with a SLIGHTLY bigger toy dataset
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [79]:
# And another to merge it with
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [84]:
ds_chars.join(states, how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


In [83]:
# Merge ds_chars and states - first an inner join
ds_chars.merge(states, how='inner',
               left_on='home_state',
               right_on='state')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [85]:
# Now an outer join
ds_chars.merge(states, how='outer',
               left_on='home_state',
               right_on='state')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [86]:
# Another mini dataset to play with
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

,cuisine,genre
0,Greek,horror
1,Indian,scifi
2,American,fantasy
3,Thai,tech
4,Indian,documentary


In [87]:
# Concat prefs and ds_chars
pd.concat([ds_chars, prefs])

,name,HP,home_state,cuisine,genre
0,greg,200.0,WA,NaN,NaN
1,miles,200.0,WA,NaN,NaN
2,alan,170.0,TX,NaN,NaN
3,alison,300.0,DC,NaN,NaN
4,rachel,200.0,TX,NaN,NaN
0,NaN,NaN,NaN,Greek,horror
1,NaN,NaN,NaN,Indian,scifi
2,NaN,NaN,NaN,American,fantasy
3,NaN,NaN,NaN,Thai,tech
4,NaN,NaN,NaN,Indian,documentary


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [90]:
# Adjust and try again
pd.concat([ds_chars, prefs], axis=1)

,name,HP,home_state,cuisine,genre
0,greg,200,WA,Greek,horror
1,miles,200,WA,Indian,scifi
2,alan,170,TX,American,fantasy
3,alison,300,DC,Thai,tech
4,rachel,200,TX,Indian,documentary


## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [91]:
# Peek at the outcomes data we already had in here
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Age in Days,Time Delta
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736,736 days
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371,371 days
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366,366 days
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128,128 days
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6,6 days


In [92]:
# Read in the intakes data
intakes= pd.read_csv('data/Austin_Animal_Center_Intakes_022822.csv')
# Check it out
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,06/29/2014 10:38:00 AM,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [96]:
# Let's try merging on Animal ID
combined = outcomes.merge(intakes, how='inner', on='Animal ID',
                          suffixes=('_outcomes', '_intakes'))

In [98]:
# What was the result?
combined.head()

,Animal ID,Name_outcomes,DateTime_outcomes,MonthYear_outcomes,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcomes,Sex upon Outcome,Age upon Outcome,...,DateTime_intakes,MonthYear_intakes,Found Location,Intake Type,Intake Condition,Animal Type_intakes,Sex upon Intake,Age upon Intake,Breed_intakes,Color_intakes
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [99]:
combined.shape

(176664, 25)

In [100]:
intakes.shape

(136763, 12)

In [101]:
outcomes.shape

(137097, 14)

Let's discuss/explore: did that work the way we expected?

- 


In [108]:
combined['Animal ID'].duplicated()

0         False
1         False
2         False
3         False
4          True
          ...  
176659    False
176660    False
176661    False
176662    False
176663    False
Name: Animal ID, Length: 176664, dtype: bool

In [111]:
# We might want to try something different
# Can we clean something to make a better merge?
combined.loc[combined.duplicated(subset='Animal ID', keep=False)]

,Animal ID,Name_outcomes,DateTime_outcomes,MonthYear_outcomes,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcomes,Sex upon Outcome,Age upon Outcome,...,DateTime_intakes,MonthYear_intakes,Found Location,Intake Type,Intake Condition,Animal Type_intakes,Sex upon Intake,Age upon Intake,Breed_intakes,Color_intakes
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
5,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
6,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
8,A659412,Princess,2020-10-05 14:37:00,Oct 2020,2013-03-24,Adoption,NaN,Dog,Spayed Female,7 years,...,06/13/2018 12:55:00 PM,June 2018,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,5 years,Chihuahua Shorthair Mix,Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176643,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,01/31/2022 05:22:00 PM,January 2022,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176644,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,02/27/2022 04:52:00 PM,February 2022,1414 South Lamar Boulevard in Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176645,A850561,Max,2022-02-05 16:22:00,Feb 2022,2020-09-27,Adoption,NaN,Dog,Neutered Male,1 year,...,01/26/2022 05:31:00 PM,January 2022,Austin (TX),Public Assist,Normal,Dog,Neutered Male,1 year,Belgian Malinois/German Shepherd,Black/Brown
176656,A850725,Monty,2022-02-25 11:23:00,Feb 2022,2020-01-30,Adoption,NaN,Dog,Neutered Male,2 years,...,01/30/2022 02:32:00 PM,January 2022,14100 Thermal Dr in Austin (TX),Stray,Injured,Dog,Intact Male,2 years,Labrador Retriever,Black/White


In [112]:
# Clean what needs cleaning...
intakes_clean = intakes.drop_duplicates(subset='Animal ID', keep='last')
outcomes_clean = outcomes.drop_duplicates(subset='Animal ID', keep='last')

In [113]:
# Try again
combined_clean = outcomes_clean.merge(intakes_clean, how='inner', on='Animal ID',
                                      suffixes=('_outcomes', '_intakes'))

In [114]:
combined_clean

,Animal ID,Name_outcomes,DateTime_outcomes,MonthYear_outcomes,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcomes,Sex upon Outcome,Age upon Outcome,...,DateTime_intakes,MonthYear_intakes,Found Location,Intake Type,Intake Condition,Animal Type_intakes,Sex upon Intake,Age upon Intake,Breed_intakes,Color_intakes
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,...,03/18/2014 07:11:00 AM,March 2014,12034 Research in Austin (TX),Stray,Nursing,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
4,A814515,Quentin,2020-05-06 07:59:00,May 2020,2018-03-01,Adoption,Foster,Dog,Neutered Male,2 years,...,03/01/2020 02:19:00 PM,March 2020,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,American Foxhound/Labrador Retriever,White/Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121768,A850166,Rainey,2022-01-24 18:20:00,Jan 2022,2021-11-19,Adoption,NaN,Cat,Intact Male,2 months,...,01/19/2022 12:39:00 PM,January 2022,South First in Austin (TX),Stray,Normal,Cat,Intact Male,1 month,Siamese,Seal Point
121769,A852031,Noodle,2022-02-28 12:50:00,Feb 2022,2020-02-23,Transfer,Partner,Dog,Neutered Male,2 years,...,02/23/2022 01:36:00 PM,February 2022,Austin (TX),Owner Surrender,Injured,Dog,Neutered Male,2 years,Pomeranian/Chihuahua Longhair,Buff
121770,A845839,*Carmen,2022-02-28 13:49:00,Feb 2022,2020-05-05,Adoption,Foster,Dog,Spayed Female,1 year,...,11/05/2021 12:19:00 PM,November 2021,12202 Von Quintus Road in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Pit Bull Mix,Brown
121771,A844321,Mia Marie,2022-02-28 13:04:00,Feb 2022,2013-10-15,Adoption,Foster,Dog,Spayed Female,8 years,...,10/15/2021 08:42:00 AM,October 2021,Manchaca And 1626 in Austin (TX),Stray,Aged,Dog,Spayed Female,8 years,Pit Bull,Black/White


# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2